This first part is commented out because I dont want to accidentally start the pulling process all over again and waste bandwidth.

In [1]:
# import requests
# datasets = requests.get('https://api.census.gov/data.json')
# jdata = datasets.json()
# df = pd.DataFrame(jdata)
# # df = pd.concat([df.drop('dataset', axis=1), df.dataset.apply(pd.Series)], axis=1)
# df = df.dataset.apply(pd.Series)
# df = df[~df.description.str.contains('phased out')]
# df.loc[df['c_isTimeseries'] == True, 'c_vintage'] = df[df['c_isTimeseries'] == True].c_vintage.fillna('Time Series')
# # vintages = df.c_vintage.value_counts().reset_index().values
# df['code'] = df.identifier.str.split('/').apply(lambda x: x[-1]).astype(str)
# df['last_year'] = df.c_vintage.astype(str)
# df.to_pickle('datasets.pkl')

This is also a pull I dont feel like repeating accidentally

In [2]:
# import pandas as pd
# import requests
# from tqdm import tqdm_notebook
# from time import sleep

# df = pd.read_pickle('datasets.pkl')
# df.c_variablesLink



# df2= pd.DataFrame()
# def variable_scrape(row):
#     title = row['title']
#     code = row['code']
#     pull = requests.get(row['c_groupsLink'])
#     global df2
#     if pull.ok:
#         backup.append(pull)
#         j = pull.json()
#         if 'variables' in j:
#             new_df = pd.DataFrame(j['variables']).T.reset_index()
#             new_df['title'] = title
#             new_df['code'] = code
#             df2 = df2.append(new_df)
#     sleep(1)

# for row in tqdm_notebook(df.index):
#     variable_scrape(df[['title', 'code', 'c_variablesLink']].loc[row])

# df2 = df2.rename({'index':'variable'}, axis=1)
# df2 = df2[['code', 'label', 'variable']]

# df2.to_pickle('variables.pkl')

In [3]:
import pandas as pd
df2 = pd.read_pickle('variables.pkl')

suffixes = ['PEA', 'PMA', 'PM', 'EA', 'MA', 'PE', 'E', 'M']

def strip_all(strip_string):
    if not type(strip_string)==str:
        return None
    for suffix in suffixes:
        if strip_string.endswith(suffix):
            if '_' in strip_string:
                return strip_string.split('_')[0]
            else:
                return strip_string.strip(suffix)
    return strip_string

df2['variable_base'] = df2['variable'].apply(strip_all)
# df.to_pickle('variable_reference.pkl')


variables_by_code = df2.groupby('code').variable_base.unique().to_dict()
# variable_labels_by_code = 

In [4]:
import pandas as pd
df1 = pd.read_pickle('datasets.pkl')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 500)
g = df1.groupby('last_year')
# codes_by_year = g.code.agg(lambda x: list(x)).to_dict()
titles_by_year = g.title.agg(lambda x: list(x)).to_dict()
# descriptions_by_year = g.description.agg(lambda x: list(x)).to_dict()
code_by_title = df1[['title', 'code']].set_index('title').to_dict()['code']
descriptions_by_title = df1[['title', 'description']].set_index('title').to_dict()['description']

vintages = list(df1.c_vintage.astype(str).unique())
vintages.sort()
vintages.reverse()

# df2 = pd.read_pickle('variable_reference.pkl')

In [5]:
data = [
    vintages,
    descriptions_by_title,
    code_by_title,
    variables_by_code,
    titles_by_year
]


import pickle
with open ('data.pkl', 'wb') as file:
    pickle.dump(data, file)
    
data_read = pickle.load(open("data.pkl", "rb"))